# Investigate content of QA pawianHists.root file

This notebook shows how to to investigate the content of a `pawianHists.root` file that is the result of the QA step in Pawian. We make use of the `pawian.qa` module of pyPawianTools.

## Draw contained histograms

A `pawianHists.root` comes with several histograms of several PWA distributions—one for data, one for the fit, and one for Monte Carlo. The `PawianHists` class contains a few methods to quickly plot these 'contained' histograms.

First, open a `pawianHists.root` file as a `PawianHists` object. In this example, we use the ROOT file that is provided in the tests of the `pawian.qa` module.

In [ ]:
from os.path import dirname, realpath
import pawian.qa as qa
from pawian.qa import PawianHists

qa_dir = dirname(realpath(qa.__file__))
filename = f'{qa_dir}/test/pawianHists_epem.root'
pawian_hists = PawianHists(filename)

Now it is quite easy to see which histograms are contained in the histogram file.

In [ ]:
histogram_names = pawian_hists.histogram_names
print(f'Number of histograms: {len(histogram_names)}\n')
print(histogram_names)

As can be seen, the names can be grouped by `Fit`, `Data`, and `MC`. The property `unique_histograme_names` helps to identify which different types there are.

In [ ]:
unique_names = pawian_hists.unique_histogram_names
print(f'Number of different histogram types: {len(unique_names)}\n')
print(unique_names)

Now, let's have a quick look at one of these histograms:

In [ ]:
import matplotlib.pyplot as plt
pawian_hists.draw_histogram('DatapionpDm')
plt.show()

This is quite ugly and, frankly speaking, not very interesting, because we actually want to **assess the quality** (QA) of our fit. Of course, you could just plot the histogram of the fit in the same figure, but that would still need some polishing to make it look nicer. And of course, you have to pay attention to use the correct names...

In [ ]:
pawian_hists.draw_histogram('DatapionpDm')
pawian_hists.draw_histogram('FitpionpDm')
plt.show()

There is an easier way to do it. For a nice comparison plot, we use the `draw_combined_histogram` method. This method works in the same way as `draw_histogram`, but it needs one of the `unique_hisogram_names`.

Note how this method can also take arguments from [`matplotlib.pyplot.hist`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) to make it look fancier, such as `density` to make the plots normalised. The histograms in the figure have been embedded with titles, so that we can nicely generate a legend as well.

Another thing to note: this time we draw the histogram on `pyplot.AxesSubplot` instead of the default `pyplot` module, as to give us the means to modify the figure a bit.

We also applied a little trick here: we used `convert` from the `pawian.latex` module to convert the histogram name to a LaTeX string.

In [ ]:
histogram_name = 'pionpDm'
fig = plt.figure(tight_layout=True, figsize=(6, 4), dpi=120).add_subplot()
pawian_hists.draw_combined_histogram(histogram_name, plot_on=fig, density=True, alpha=.5)
plt.ylim(bottom=0)
plt.legend()

from pawian.latex import convert
plt.xlabel(f'$M({convert(histogram_name)}$)')
plt.ylabel('counts')
plt.show()

Finally, a thing you may want to have done immediately is to generate an overview of all histograms. This can be done with the `draw_all_histograms` method. It takes some time to draw them all, but it's worth it!

By the way, notice how the Monte Carlo samples have been hidden by using `mc=False`. You can do the same with `data=False` and/or `fit=False`.

This method again takes arguments from [`matplotlib.pyplot.hist`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html), so feel free to play around and modify the figures.

In [ ]:
fig = plt.figure(tight_layout=True, figsize=(15, 14))
pawian_hists.draw_all_histograms(mc=False,
    plot_on=fig, legend='upper right', alpha=.5, density=True)
plt.show()

## Plot vector distributions

A `pawianHists.root` file also contains trees with the original Lorentz vectors of the data and of the Monte Carlo sample. You can also easily access these vectors from the `PawianHists` object.

`PawianHists` objects contain an `EventSet` for data and for fitted data. In turn, each `EventSet` contains an event-based of weights and a dictionary of Lorentz vectors. Each entry in the dictionary represents one particle.

In [ ]:
print('Contains particles:', pawian_hists.data.particles)
print('Average data weight:', pawian_hists.data.weights.mean())
print('Number of data events:', len(pawian_hists.data))
print('Number of MC events:', len(pawian_hists.fit))

We are interested in those arrays of Lorentz vectors. These can be obtained by calling an index on either the `data` or `fit` member. Then it's merely a matter of calling the methods of the [`TLorentzVector` class](https://github.com/scikit-hep/uproot-methods/blob/master/uproot_methods/classes/TLorentzVector.py) of `uproot_methods` to obtain observables in which you are interested.

In [ ]:
pions = pawian_hists.data['pion+']
Dm = pawian_hists.data['D-']
D0 = pawian_hists.data['D0']

for particle, vectors in pawian_hists.data.items():
    print(f'Mean mass {particle:<5} = {vectors.mass.mean():7.5} ± {vectors.mass.std():5.3}')

Note also that these `TLorentzVector` arrays can be added and then plotted in a histogram!

In [ ]:
piDm = pions + Dm
fig1, fig2 = plt.figure(figsize=(15, 5)).subplots(1, 2)
fig1.hist(piDm.mass, bins=30)
fig1.set_title('$M(\pi^+D^-)$')
fig2.hist(piDm.perp, bins=30)
fig2.set_title('$|p_{\pi^+D^-}|$')
plt.show()

The [`TLorentzVector` class](https://github.com/scikit-hep/uproot-methods/blob/master/uproot_methods/classes/TLorentzVector.py) even has methods like `rotatey` and `boost`, effecticely allowing you to compute the relevant kinematic variables to do PWA. All you have to do is call these methods on the (added) arrays) illustrated above!